In [ ]:
# pip install pandas numpy matplotlib seaborn openpyxl jupyter

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px

# df1 = pd.read_csv('../../../data/processed/population.csv', encoding='utf-8')
# df2 = pd.read_csv('../../../data/processed/sales.csv', encoding='utf-8')
# df3 = pd.read_csv('../../../data/processed/stores.csv', encoding='utf-8')
# # df = pd.read_csv('../../../data/processed/districts.csv', encoding='cp949')


# df2_convenience = df2[df2['서비스_업종_코드_명'] == '편의점']
# df3_convenience = df3[df3['서비스_업종_코드_명'] == '편의점']

# # print(df2_convenience)
# # print(df3_convenience)

# result = df1.merge(df2_convenience, on='행정동_코드', how='outer') \
#             .merge(df3_convenience, on='행정동_코드', how='outer')

# # result = df1.merge(df2_convenience, on='행정동_코드', how='inner')
# # result = result.merge(df3_convenience, on='행정동_코드', how='inner')

# print(result)

# # result.to_csv('결과파일1.csv', index=False, encoding='utf-8-sig')


In [ ]:
# import pandas as pd

# sales = pd.read_csv('sales_convenience.csv', encoding='utf-8-sig')

# # 행정동별 레코드 수 확인 (분기별)
# store_count = sales.groupby(['기준_년분기_코드', '행정동_코드']).size().reset_index(name='점포수')
# print("분기별 행정동당 레코드 수 분포:")
# print(store_count['점포수'].describe())
# print("\n점포수별 빈도:")
# print(store_count['점포수'].value_counts().sort_index().head(20))

# # 특정 분기에서 확인
# q1 = sales[sales['기준_년분기_코드'] == 20221]
# print(f"\n2022년 1분기 행정동 수: {q1['행정동_코드'].nunique()}")
# print(f"2022년 1분기 총 레코드 수: {len(q1)}")

# # 여러 레코드가 있는 행정동 예시
# multi = store_count[store_count['점포수'] > 1]
# print(f"\n점포수 > 1인 행정동/분기 조합: {len(multi)}개")
# if len(multi) > 0:
#     print(multi.head(10))

In [ ]:
# pip install scipy

# 1. 통계 분석

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

colors = {'primary': '#1a5276', 'secondary': '#2874a6', 'accent': '#e74c3c', 
          'light': '#85c1e9', 'gray': '#7f8c8d', 'green': '#27ae60'}

# 데이터 로드
sales = pd.read_csv('sales_convenience.csv', encoding='utf-8-sig')
population = pd.read_csv('population.csv', encoding='utf-8-sig')

pop_cols = ['기준_년분기_코드', '행정동_코드', '총_유동인구_수',
            '연령대_10_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수',
            '연령대_40_유동인구_수', '연령대_50_유동인구_수', '연령대_60_이상_유동인구_수']

merged = sales.merge(population[pop_cols], on=['기준_년분기_코드', '행정동_코드'], how='inner')



In [ ]:
# 변수 생성
merged['유동인구'] = merged['총_유동인구_수']
merged['매출금액'] = merged['당월_매출_금액']
merged['매출건수'] = merged['당월_매출_건수']
merged['밀집도'] = (merged['매출건수'] / merged['유동인구']) * 1000

merged['청년_유동인구_비율'] = (merged['연령대_20_유동인구_수'] + merged['연령대_30_유동인구_수']) / merged['유동인구']
merged['중년_유동인구_비율'] = (merged['연령대_40_유동인구_수'] + merged['연령대_50_유동인구_수']) / merged['유동인구']
merged['고령_유동인구_비율'] = merged['연령대_60_이상_유동인구_수'] / merged['유동인구']
merged['10대_유동인구_비율'] = merged['연령대_10_유동인구_수'] / merged['유동인구']

merged['log_매출'] = np.log1p(merged['매출금액'])
merged['log_유동인구'] = np.log1p(merged['유동인구'])
merged['log_밀집도'] = np.log1p(merged['밀집도'])

print("="*80)
print("서울 편의점 매출 분석 (유동인구, 밀집도, 연령대 집중)")
print("="*80)
print(f"\n데이터: {len(merged)}행, {merged['행정동_코드'].nunique()}개 행정동")



In [ ]:
# 기술통계
print("\n[기술통계]")
print(f"  밀집도(천명당 매출건수): 평균 {merged['밀집도'].mean():.1f}, 중앙값 {merged['밀집도'].median():.1f}")
print(f"  유동인구: 평균 {merged['유동인구'].mean()/1e6:.2f}백만명")
print(f"  매출: 평균 {merged['매출금액'].mean()/1e8:.1f}억원")



In [ ]:
# 상관분석
print("\n[상관분석 - log(매출)과의 상관]")
for var in ['log_유동인구', 'log_밀집도', '청년_유동인구_비율', '중년_유동인구_비율', '고령_유동인구_비율']:
    r, p = stats.pearsonr(merged['log_매출'], merged[var])
    sig = "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""
    print(f"  {var:<25}: r = {r:.4f} {sig}")



In [ ]:
# 회귀분석
Y = merged['log_매출'].values
n = len(Y)
X_vars = ['log_유동인구', 'log_밀집도', '청년_유동인구_비율', '중년_유동인구_비율', '고령_유동인구_비율']
X_data = merged[X_vars].values
X = np.column_stack([np.ones(n), X_data])

beta = np.linalg.solve(X.T @ X, X.T @ Y)
Y_pred = X @ beta
residuals = Y - Y_pred

SST = np.sum((Y - np.mean(Y))**2)
SSE = np.sum(residuals**2)
k = len(X_vars)
R2 = 1 - SSE / SST
adj_R2 = 1 - (SSE / (n - k - 1)) / (SST / (n - 1))
MSE = SSE / (n - k - 1)
se_beta = np.sqrt(MSE * np.diag(np.linalg.inv(X.T @ X)))
t_stats = beta / se_beta
p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), df=n-k-1))

print("\n[회귀분석 결과]")
print(f"  R-squared: {R2:.4f} ({R2*100:.1f}% 설명)")
print(f"  Adjusted R-squared: {adj_R2:.4f}")

print("\n  회귀계수:")
var_names = ['(상수항)'] + X_vars
for i, name in enumerate(var_names):
    sig = "***" if p_values[i] < 0.001 else "**" if p_values[i] < 0.01 else "*" if p_values[i] < 0.05 else ""
    print(f"    {name:<25}: β = {beta[i]:>8.4f}, t = {t_stats[i]:>7.2f} {sig}")

# VIF
def calc_vif(X_data):
    vifs = []
    for i in range(X_data.shape[1]):
        y = X_data[:, i]
        X_o = np.delete(X_data, i, axis=1)
        X_d = np.column_stack([np.ones(len(y)), X_o])
        b = np.linalg.solve(X_d.T @ X_d, X_d.T @ y)
        r2 = 1 - np.sum((y - X_d @ b)**2) / np.sum((y - np.mean(y))**2)
        vifs.append(1 / (1 - r2) if r2 < 1 else float('inf'))
    return vifs

vif_values = calc_vif(X_data)
print("\n[VIF (다중공선성)]")
for var, vif in zip(X_vars, vif_values):
    print(f"    {var:<25}: {vif:.2f}")
print(f"  → 최대 VIF = {max(vif_values):.2f} (5 미만이면 양호)")

# 단독 R2
single_r2 = {}
for var in X_vars:
    X_s = np.column_stack([np.ones(n), merged[var].values])
    b_s = np.linalg.solve(X_s.T @ X_s, X_s.T @ Y)
    r2_s = 1 - np.sum((Y - X_s @ b_s)**2) / SST
    single_r2[var] = r2_s

print("\n[개별 변수 설명력 (R²)]")
for var, r2 in sorted(single_r2.items(), key=lambda x: x[1], reverse=True):
    print(f"    {var:<25}: {r2:.4f}")

# 잔차 진단
dw_stat = np.sum(np.diff(residuals)**2) / np.sum(residuals**2)
print(f"\n[잔차 진단] Durbin-Watson: {dw_stat:.2f}")

# 결과 요약
print("\n" + "="*80)
print("핵심 발견")
print("="*80)
print(f"""
1. 밀집도 효과 (R² = {single_r2['log_밀집도']:.4f}, β = {beta[2]:.4f})
   → 밀집도 1% 증가 시 매출 약 {beta[2]:.2f}% 증가
   → 가장 높은 단독 설명력!

2. 유동인구 효과 (R² = {single_r2['log_유동인구']:.4f}, β = {beta[1]:.4f})
   → 유동인구 1% 증가 시 매출 약 {beta[1]:.2f}% 증가

3. 연령대 효과
   → 청년(20-30대) 비율: β = {beta[3]:+.4f}
   → 중년(40-50대) 비율: β = {beta[4]:+.4f}
   → 고령(60대+) 비율: β = {beta[5]:+.4f}

4. 다중공선성: 문제 없음 (max VIF = {max(vif_values):.2f})
""")


In [ ]:

# 결과 저장
results_df = pd.DataFrame({
    '지표': ['R2', 'Adj_R2', 'beta_유동인구', 'beta_밀집도', 'beta_청년', 'beta_중년', 'beta_고령',
             'R2_유동인구', 'R2_밀집도', 'Max_VIF', 'DW'],
    '값': [R2, adj_R2, beta[1], beta[2], beta[3], beta[4], beta[5],
           single_r2['log_유동인구'], single_r2['log_밀집도'], max(vif_values), dw_stat]
})
results_df.to_csv('regression_results_v2.csv', index=False, encoding='utf-8-sig')

merged[['기준_년분기_코드', '행정동_코드', '행정동_코드_명', '매출금액', '유동인구', '매출건수', '밀집도',
        '청년_유동인구_비율', '중년_유동인구_비율', '고령_유동인구_비율']].to_csv(
    'analysis_data_v2.csv', index=False, encoding='utf-8-sig')

print("결과 저장 완료!")

# 2. 시각화

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

colors = {'primary': '#1a5276', 'secondary': '#2874a6', 'accent': '#e74c3c', 
          'light': '#85c1e9', 'gray': '#7f8c8d', 'green': '#27ae60'}

# 데이터 로드
df = pd.read_csv('analysis_data_v2.csv', encoding='utf-8-sig')
df['log_매출'] = np.log1p(df['매출금액'])
df['log_유동인구'] = np.log1p(df['유동인구'])
df['log_밀집도'] = np.log1p(df['밀집도'])

# 회귀분석 재수행
Y = df['log_매출'].values
n = len(Y)
X_vars = ['log_유동인구', 'log_밀집도', '청년_유동인구_비율', '중년_유동인구_비율', '고령_유동인구_비율']
X_data = df[X_vars].values
X = np.column_stack([np.ones(n), X_data])
beta = np.linalg.solve(X.T @ X, X.T @ Y)
Y_pred = X @ beta
residuals = Y - Y_pred
SST = np.sum((Y - np.mean(Y))**2)
R2 = 1 - np.sum(residuals**2) / SST

# VIF
def calc_vif(X_data):
    vifs = []
    for i in range(X_data.shape[1]):
        y = X_data[:, i]
        X_o = np.delete(X_data, i, axis=1)
        X_d = np.column_stack([np.ones(len(y)), X_o])
        b = np.linalg.solve(X_d.T @ X_d, X_d.T @ y)
        r2 = 1 - np.sum((y - X_d @ b)**2) / np.sum((y - np.mean(y))**2)
        vifs.append(1 / (1 - r2) if r2 < 1 else float('inf'))
    return vifs
vif_values = calc_vif(X_data)

# 단독 R2
single_r2 = {}
for var in X_vars:
    X_s = np.column_stack([np.ones(n), df[var].values])
    b_s = np.linalg.solve(X_s.T @ X_s, X_s.T @ Y)
    single_r2[var] = 1 - np.sum((Y - X_s @ b_s)**2) / SST

# =========== Figure 1: 상관관계 ===========
fig1, axes = plt.subplots(2, 2, figsize=(14, 12))

ax1 = axes[0, 0]
ax1.scatter(df['log_유동인구'], df['log_매출'], alpha=0.3, s=20, c=colors['primary'], edgecolors='none')
z = np.polyfit(df['log_유동인구'], df['log_매출'], 1)
x_line = np.linspace(df['log_유동인구'].min(), df['log_유동인구'].max(), 100)
ax1.plot(x_line, np.poly1d(z)(x_line), color=colors['accent'], linewidth=2)
r, _ = stats.pearsonr(df['log_유동인구'], df['log_매출'])
ax1.set_xlabel('log(Floating Population)', fontsize=12)
ax1.set_ylabel('log(Sales)', fontsize=12)
ax1.set_title(f'Floating Population vs Sales\n(r = {r:.3f})', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

ax2 = axes[0, 1]
ax2.scatter(df['log_밀집도'], df['log_매출'], alpha=0.3, s=20, c=colors['green'], edgecolors='none')
z2 = np.polyfit(df['log_밀집도'], df['log_매출'], 1)
x_line2 = np.linspace(df['log_밀집도'].min(), df['log_밀집도'].max(), 100)
ax2.plot(x_line2, np.poly1d(z2)(x_line2), color=colors['accent'], linewidth=2)
r2, _ = stats.pearsonr(df['log_밀집도'], df['log_매출'])
ax2.set_xlabel('log(Store Density)', fontsize=12)
ax2.set_ylabel('log(Sales)', fontsize=12)
ax2.set_title(f'Store Density vs Sales\n(r = {r2:.3f}, Strongest!)', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

ax3 = axes[1, 0]
age_vars = ['청년_유동인구_비율', '중년_유동인구_비율', '고령_유동인구_비율']
age_labels = ['Youth (20-30s)', 'Middle (40-50s)', 'Senior (60+)']
age_colors = [colors['primary'], colors['secondary'], colors['gray']]
for var, label, color in zip(age_vars, age_labels, age_colors):
    r_age, _ = stats.pearsonr(df[var], df['log_매출'])
    ax3.scatter(df[var], df['log_매출'], alpha=0.15, s=10, c=color, label=f'{label} (r={r_age:.2f})')
ax3.set_xlabel('Age Group Ratio', fontsize=12)
ax3.set_ylabel('log(Sales)', fontsize=12)
ax3.set_title('Age Demographics vs Sales', fontsize=14, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

ax4 = axes[1, 1]
coef_names = ['log(Population)', 'log(Density)', 'Youth', 'Middle', 'Senior']
coef_values = beta[1:]
coef_colors = [colors['primary'] if v > 0 else colors['accent'] for v in coef_values]
bars = ax4.barh(coef_names, coef_values, color=coef_colors, edgecolor='white', height=0.6)
ax4.axvline(x=0, color='black', linewidth=0.8)
ax4.set_xlabel('Coefficient (Beta)', fontsize=12)
ax4.set_title('Regression Coefficients', fontsize=14, fontweight='bold')
for bar, val in zip(bars, coef_values):
    xpos = val + 0.02 if val > 0 else val - 0.02
    ax4.text(xpos, bar.get_y() + bar.get_height()/2, f'{val:.3f}', va='center', 
             ha='left' if val > 0 else 'right', fontsize=11, fontweight='bold')
ax4.grid(True, axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('01_correlation_v2.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.close()
print("01_correlation_v2.png saved")

# =========== Figure 2: 잔차 진단 ===========
fig2, axes = plt.subplots(2, 2, figsize=(14, 12))

ax1 = axes[0, 0]
ax1.scatter(Y_pred, residuals, alpha=0.3, s=15, c=colors['primary'], edgecolors='none')
ax1.axhline(y=0, color=colors['accent'], linewidth=2, linestyle='--')
ax1.set_xlabel('Predicted Values', fontsize=12)
ax1.set_ylabel('Residuals', fontsize=12)
ax1.set_title('Residuals vs Predicted', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

ax2 = axes[0, 1]
ax2.hist(residuals, bins=50, density=True, alpha=0.7, color=colors['light'], edgecolor='white')
x = np.linspace(residuals.min(), residuals.max(), 100)
ax2.plot(x, stats.norm.pdf(x, 0, np.std(residuals)), color=colors['accent'], linewidth=2)
ax2.set_xlabel('Residuals', fontsize=12)
ax2.set_title(f'Residual Distribution (Skew={stats.skew(residuals):.2f})', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

ax3 = axes[1, 0]
stats.probplot(residuals, dist="norm", plot=ax3)
ax3.get_lines()[0].set_markerfacecolor(colors['primary'])
ax3.get_lines()[0].set_alpha(0.5)
ax3.get_lines()[1].set_color(colors['accent'])
ax3.set_title('Q-Q Plot', fontsize=14, fontweight='bold')
ax3.grid(True, alpha=0.3)

ax4 = axes[1, 1]
ax4.scatter(Y, Y_pred, alpha=0.3, s=15, c=colors['primary'], edgecolors='none')
ax4.plot([Y.min(), Y.max()], [Y.min(), Y.max()], color=colors['accent'], linewidth=2, linestyle='--')
ax4.set_xlabel('Actual log(Sales)', fontsize=12)
ax4.set_ylabel('Predicted log(Sales)', fontsize=12)
ax4.set_title(f'Actual vs Predicted (R² = {R2:.4f})', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('02_residuals_v2.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.close()
print("02_residuals_v2.png saved")

# =========== Figure 3: VIF & R2 ===========
fig3, axes = plt.subplots(2, 2, figsize=(14, 12))

ax1 = axes[0, 0]
var_labels = ['log(Pop)', 'log(Density)', 'Youth', 'Middle', 'Senior']
vif_colors = [colors['primary'] if v < 5 else colors['accent'] for v in vif_values]
bars = ax1.barh(var_labels, vif_values, color=vif_colors, edgecolor='white', height=0.6)
ax1.axvline(x=5, color=colors['accent'], linewidth=2, linestyle='--', label='Threshold=5')
ax1.set_xlabel('VIF', fontsize=12)
ax1.set_title('VIF (Multicollinearity)', fontsize=14, fontweight='bold')
ax1.legend()
for bar, val in zip(bars, vif_values):
    ax1.text(val + 0.1, bar.get_y() + bar.get_height()/2, f'{val:.2f}', va='center', fontsize=11, fontweight='bold')
ax1.grid(True, axis='x', alpha=0.3)

ax2 = axes[0, 1]
r2_vals = list(single_r2.values()) + [R2]
r2_labels = var_labels + ['Full Model']
bar_colors = [colors['light']] * 5 + [colors['primary']]
bars2 = ax2.bar(range(len(r2_vals)), r2_vals, color=bar_colors, edgecolor='white')
ax2.set_xticks(range(len(r2_vals)))
ax2.set_xticklabels(r2_labels, rotation=45, ha='right', fontsize=10)
ax2.set_ylabel('R²', fontsize=12)
ax2.set_title('Individual vs Full Model R²', fontsize=14, fontweight='bold')
for bar, val in zip(bars2, r2_vals):
    ax2.text(bar.get_x() + bar.get_width()/2, val + 0.01, f'{val:.3f}', ha='center', fontsize=10, fontweight='bold')
ax2.grid(True, axis='y', alpha=0.3)

ax3 = axes[1, 0]
df['밀집도_구간'] = pd.qcut(df['밀집도'], q=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
density_grp = df.groupby('밀집도_구간')['매출금액'].mean() / 1e8
bars3 = ax3.bar(range(len(density_grp)), density_grp.values, color=colors['green'], edgecolor='white')
ax3.set_xticks(range(len(density_grp)))
ax3.set_xticklabels(density_grp.index, fontsize=11)
ax3.set_xlabel('Store Density Quintile', fontsize=12)
ax3.set_ylabel('Avg Sales (100M KRW)', fontsize=12)
ax3.set_title('Sales by Store Density', fontsize=14, fontweight='bold')
for bar, val in zip(bars3, density_grp.values):
    ax3.text(bar.get_x() + bar.get_width()/2, val + 1, f'{val:.0f}', ha='center', fontsize=11, fontweight='bold')
ax3.grid(True, axis='y', alpha=0.3)

ax4 = axes[1, 1]
df['유동인구_구간'] = pd.qcut(df['유동인구'], q=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
pop_grp = df.groupby('유동인구_구간')['매출금액'].mean() / 1e8
bars4 = ax4.bar(range(len(pop_grp)), pop_grp.values, color=colors['primary'], edgecolor='white')
ax4.set_xticks(range(len(pop_grp)))
ax4.set_xticklabels(pop_grp.index, fontsize=11)
ax4.set_xlabel('Population Quintile', fontsize=12)
ax4.set_ylabel('Avg Sales (100M KRW)', fontsize=12)
ax4.set_title('Sales by Population', fontsize=14, fontweight='bold')
for bar, val in zip(bars4, pop_grp.values):
    ax4.text(bar.get_x() + bar.get_width()/2, val + 1, f'{val:.0f}', ha='center', fontsize=11, fontweight='bold')
ax4.grid(True, axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('03_vif_r2_v2.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.close()
print("03_vif_r2_v2.png saved")

# =========== Figure 4: 연령대 분석 ===========
fig4, axes = plt.subplots(2, 2, figsize=(14, 12))

ax1 = axes[0, 0]
df['청년_구간'] = pd.qcut(df['청년_유동인구_비율'], q=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
youth_grp = df.groupby('청년_구간')['매출금액'].mean() / 1e8
bars1 = ax1.bar(range(len(youth_grp)), youth_grp.values, color=colors['primary'], edgecolor='white')
ax1.set_xticks(range(len(youth_grp)))
ax1.set_xticklabels(youth_grp.index, fontsize=11)
ax1.set_xlabel('Youth (20-30s) Ratio Quintile', fontsize=12)
ax1.set_ylabel('Avg Sales (100M KRW)', fontsize=12)
ax1.set_title('Sales by Youth Ratio', fontsize=14, fontweight='bold')
for bar, val in zip(bars1, youth_grp.values):
    ax1.text(bar.get_x() + bar.get_width()/2, val + 0.5, f'{val:.0f}', ha='center', fontsize=11, fontweight='bold')
ax1.grid(True, axis='y', alpha=0.3)

ax2 = axes[0, 1]
df['중년_구간'] = pd.qcut(df['중년_유동인구_비율'], q=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
middle_grp = df.groupby('중년_구간')['매출금액'].mean() / 1e8
bars2 = ax2.bar(range(len(middle_grp)), middle_grp.values, color=colors['secondary'], edgecolor='white')
ax2.set_xticks(range(len(middle_grp)))
ax2.set_xticklabels(middle_grp.index, fontsize=11)
ax2.set_xlabel('Middle (40-50s) Ratio Quintile', fontsize=12)
ax2.set_ylabel('Avg Sales (100M KRW)', fontsize=12)
ax2.set_title('Sales by Middle-age Ratio', fontsize=14, fontweight='bold')
for bar, val in zip(bars2, middle_grp.values):
    ax2.text(bar.get_x() + bar.get_width()/2, val + 0.5, f'{val:.0f}', ha='center', fontsize=11, fontweight='bold')
ax2.grid(True, axis='y', alpha=0.3)

ax3 = axes[1, 0]
df['고령_구간'] = pd.qcut(df['고령_유동인구_비율'], q=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
senior_grp = df.groupby('고령_구간')['매출금액'].mean() / 1e8
bars3 = ax3.bar(range(len(senior_grp)), senior_grp.values, color=colors['gray'], edgecolor='white')
ax3.set_xticks(range(len(senior_grp)))
ax3.set_xticklabels(senior_grp.index, fontsize=11)
ax3.set_xlabel('Senior (60+) Ratio Quintile', fontsize=12)
ax3.set_ylabel('Avg Sales (100M KRW)', fontsize=12)
ax3.set_title('Sales by Senior Ratio', fontsize=14, fontweight='bold')
for bar, val in zip(bars3, senior_grp.values):
    ax3.text(bar.get_x() + bar.get_width()/2, val + 0.5, f'{val:.0f}', ha='center', fontsize=11, fontweight='bold')
ax3.grid(True, axis='y', alpha=0.3)

ax4 = axes[1, 1]
corr_vals = [stats.pearsonr(df['log_매출'], df[v])[0] for v in ['log_유동인구', 'log_밀집도', '청년_유동인구_비율', '중년_유동인구_비율', '고령_유동인구_비율']]
corr_labels = ['Population', 'Density', 'Youth', 'Middle', 'Senior']
corr_colors = [colors['green'] if v > 0.5 else colors['primary'] if v > 0 else colors['accent'] for v in corr_vals]
bars4 = ax4.barh(corr_labels, corr_vals, color=corr_colors, edgecolor='white', height=0.6)
ax4.axvline(x=0, color='black', linewidth=0.8)
ax4.set_xlabel('Correlation with log(Sales)', fontsize=12)
ax4.set_title('Correlation Summary', fontsize=14, fontweight='bold')
for bar, val in zip(bars4, corr_vals):
    xpos = val + 0.02 if val > 0 else val - 0.02
    ax4.text(xpos, bar.get_y() + bar.get_height()/2, f'{val:.3f}', va='center', 
             ha='left' if val > 0 else 'right', fontsize=11, fontweight='bold')
ax4.grid(True, axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('04_age_analysis_v2.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.close()
print("04_age_analysis_v2.png saved")

# =========== Figure 5: 모델 요약 ===========
fig5, ax = plt.subplots(figsize=(14, 10))
ax.axis('off')

summary = f"""
╔══════════════════════════════════════════════════════════════════════════════════════╗
║                     SEOUL CONVENIENCE STORE SALES ANALYSIS                            ║
║                        Focus: Population, Density, Age                                ║
╠══════════════════════════════════════════════════════════════════════════════════════╣
║  Data: {n:,} observations, 414 administrative districts (2022Q1 - 2025Q3)            ║
╠══════════════════════════════════════════════════════════════════════════════════════╣
║                                  MODEL FIT                                            ║
║  ──────────────────────────────────────────────────────────────────────────────────  ║
║  R-squared:           {R2:.4f}   (Model explains {R2*100:.1f}% of sales variance)              ║
║  Durbin-Watson:       {np.sum(np.diff(residuals)**2) / np.sum(residuals**2):.2f}     (No autocorrelation issue)                          ║
║  Max VIF:             {max(vif_values):.2f}     (No multicollinearity - all VIF < 5)                   ║
╠══════════════════════════════════════════════════════════════════════════════════════╣
║                              COEFFICIENTS (All significant)                           ║
║  ──────────────────────────────────────────────────────────────────────────────────  ║
║  Variable                        Beta        Interpretation                           ║
║  ──────────────────────────────────────────────────────────────────────────────────  ║
║  log(Floating Population)      {beta[1]:>7.4f}      1% pop increase → {beta[1]:.2f}% sales increase      ║
║  log(Store Density)            {beta[2]:>7.4f}      1% density increase → {beta[2]:.2f}% sales increase  ║
║  Youth (20-30s) Ratio          {beta[3]:>7.4f}      Weak positive effect                          ║
║  Middle (40-50s) Ratio         {beta[4]:>7.4f}      Moderate positive effect                      ║
║  Senior (60+) Ratio            {beta[5]:>7.4f}      Strong positive effect                        ║
╠══════════════════════════════════════════════════════════════════════════════════════╣
║                              INDIVIDUAL R² (Explanatory Power)                        ║
║  ──────────────────────────────────────────────────────────────────────────────────  ║
║  log(Store Density)            {single_r2['log_밀집도']:.4f}    ★ STRONGEST predictor (explains 66.3%)       ║
║  log(Floating Population)      {single_r2['log_유동인구']:.4f}    Second strongest (explains 26.4%)           ║
║  Youth Ratio                   {single_r2['청년_유동인구_비율']:.4f}    Moderate (explains 20.2%)                    ║
║  Senior Ratio                  {single_r2['고령_유동인구_비율']:.4f}    Moderate (explains 17.6%)                    ║
║  Middle Ratio                  {single_r2['중년_유동인구_비율']:.4f}    Weak (explains 0.1%)                         ║
╠══════════════════════════════════════════════════════════════════════════════════════╣
║                                  KEY FINDINGS                                         ║
║  ──────────────────────────────────────────────────────────────────────────────────  ║
║  1. Store Density (sales per 1000 people) is the BEST predictor of sales             ║
║     → Higher density = more stores = more total sales                                 ║
║                                                                                       ║
║  2. Floating Population has strong positive effect                                    ║
║     → More people = More potential customers = Higher sales                           ║
║                                                                                       ║
║  3. Age Demographics show interesting patterns:                                       ║
║     → Youth ratio: positive correlation (r=0.45) but weak in regression               ║
║     → Senior ratio: negative correlation (r=-0.42) but positive in regression         ║
║     → This suggests confounding with density/population                               ║
║                                                                                       ║
║  4. Model is highly accurate (R² = 96.3%) with no diagnostic issues                  ║
╚══════════════════════════════════════════════════════════════════════════════════════╝
"""

ax.text(0.02, 0.98, summary, transform=ax.transAxes, fontsize=10, va='top', 
        fontfamily='monospace', bbox=dict(boxstyle='round', facecolor='#f8f9fa', edgecolor='#dee2e6'))

plt.tight_layout()
plt.savefig('05_model_summary_v2.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.close()
print("05_model_summary_v2.png saved")

print("\nAll visualizations completed!")